In [10]:
import datetime
import time
import os
import pandas as pd
import json
totalRuns=3
endpoint = "http://localhost:9090/api/v1/query?query="
total_requests_query = f"{endpoint}sum(istio_requests_total{{namespace=~'sock-shop|hawk-ns'}})[1h:1s]"
total_requests_collector_query = f"{endpoint}sum(istio_requests_total{{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'}})[1h:1s]"
latency_ms_collector_query = f"{endpoint}(sum(increase(istio_request_duration_milliseconds_sum{{destination_app='collector'}}[1h]))/sum(increase(istio_request_duration_milliseconds_count{{destination_app='collector'}}[1h])))[1h:1s]"

def get_prometheus_query(r,download_filename,first,last,query)->pd.Series:
    filename = f"testresults/{r}_{download_filename}"
    os.system(f'curl -o {filename} -g "{query}"')
    print(query)
    with open(filename, 'r') as f:
        data = json.load(f)["data"]["result"][0]["values"]
    df = pd.DataFrame(data).convert_dtypes().set_index(0,drop=True)
    
    return pd.to_numeric(df.loc[first:last][1])

In [17]:
for r in range(totalRuns):
    print(f"Run: {r}/{totalRuns}")
    os.system(f"locust --csv=testresults/{r}")
    hist_df = pd.read_csv(f"testresults/{r}_stats_history.csv")
    first = int(hist_df["Timestamp"].head(1).values[0])
    last = int(hist_df["Timestamp"].tail(1).values[0])
    hist_df.set_index("Timestamp",inplace=True)
    print("Sleep 10 seconds...")

Run: 0/3


[2023-03-03 10:25:26,179] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:25:26,182] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:25:26,183] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...
Run: 1/3


[2023-03-03 10:26:27,943] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:26:27,946] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:26:27,946] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...
Run: 2/3


[2023-03-03 10:27:30,346] ciph/WARNING/locust.main: System open file limit '8192' is below minimum setting '10000'.
It's not high enough for load testing, and the OS didn't allow locust to increase it by itself.
See https://github.com/locustio/locust/wiki/Installation#increasing-maximum-number-of-open-files-limit for more info.
[2023-03-03 10:27:30,350] ciph/INFO/locust.main: Run time limit set to 61 seconds
[2023-03-03 10:27:30,350] ciph/INFO/locust.main: Starting Locust 2.15.0
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
--------|----------------------------------------------------------------------------|-------|-------------|-------|-------|-------|-------|--------|-----------
         Aggregated              

Sleep 10 seconds...


In [18]:
hist_df = pd.read_csv(f"testresults/{0}_stats_history.csv")
first = int(hist_df["Timestamp"].head(1).values[0])
last = int(hist_df["Timestamp"].tail(1).values[0])
hist_df.set_index("Timestamp",inplace=True)

In [19]:
hist_df["Istio_Requests_Total"] = get_prometheus_query(0,"istio_total_requests.json",first,last,total_requests_query)

http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns'})[1h:1s]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 43638    0 43638    0     0  1947k      0 --:--:-- --:--:-- --:--:-- 2029k


In [20]:
hist_df["Istio_Requests_Collector_Total"] = get_prometheus_query(0,"istio_total_requests_collector.json",first,last,total_requests_collector_query)

http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'})[1h:1s]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  8422    0  8422    0     0   560k      0 --:--:-- --:--:-- --:--:--  587k


In [21]:
hist_df["Collector_ms_Latency"] = get_prometheus_query(r,"collector_latency.json",first,last,latency_ms_collector_query)

http://localhost:9090/api/v1/query?query=(sum(increase(istio_request_duration_milliseconds_sum{destination_app='collector'}[1h]))/sum(increase(istio_request_duration_milliseconds_count{destination_app='collector'}[1h])))[1h:1s]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 14644    0 14644    0     0   722k      0 --:--:-- --:--:-- --:--:--  752k


In [22]:
hist_df.to_csv(f"testresults/{r}_combined.csv")

In [23]:
for r in range(totalRuns):
    hist_df = pd.read_csv(f"testresults/{r}_stats_history.csv")
    first = int(hist_df["Timestamp"].head(1).values[0])
    last = int(hist_df["Timestamp"].tail(1).values[0])
    hist_df.set_index("Timestamp",inplace=True)
    hist_df["Istio_Requests_Total"] = get_prometheus_query(r,"istio_total_requests.json",first,last,total_requests_query)
    hist_df["Istio_Requests_Collector_Total"] = get_prometheus_query(r,"istio_total_requests_collector.json",first,last,total_requests_collector_query)
    hist_df["Collector_ms_Latency"] = get_prometheus_query(r,"collector_latency.json",first,last,latency_ms_collector_query)
    hist_df.to_csv(f"testresults/{r}_combined.csv")

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48027    0 48027    0     0  1239k      0 --:--:-- --:--:-- --:--:-- 1267k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12562    0 12562    0     0   563k      0 --:--:-- --:--:-- --:--:--  613k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20844    0 20844    0     0   714k      0 --:--:-- --:--:-- --:--:--  753k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns'})[1h:1s]
http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'})[1h:1s]
http://localhost:9090/api/v1/query?query=(sum(increase(istio_request_duration_milliseconds_sum{destination_app='collector'}[1h]))/sum(increase(istio_request_duration_milliseconds_count{destination_app='collector'}[1h])))[1h:1s]
http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns'})[1h:1s]


100 48048    0 48048    0     0  1536k      0 --:--:-- --:--:-- --:--:-- 1617k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12582    0 12582    0     0   507k      0 --:--:-- --:--:-- --:--:--  558k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20844    0 20844    0     0   469k      0 --:--:-- --:--:-- --:--:--  484k


http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'})[1h:1s]
http://localhost:9090/api/v1/query?query=(sum(increase(istio_request_duration_milliseconds_sum{destination_app='collector'}[1h]))/sum(increase(istio_request_duration_milliseconds_count{destination_app='collector'}[1h])))[1h:1s]
http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns'})[1h:1s]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 48048    0 48048    0     0  1381k      0 --:--:-- --:--:-- --:--:-- 1380k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 12582    0 12582    0     0   308k      0 --:--:-- --:--:-- --:--:--  315k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0

http://localhost:9090/api/v1/query?query=sum(istio_requests_total{namespace=~'sock-shop|hawk-ns',destination_service_name='collector'})[1h:1s]
http://localhost:9090/api/v1/query?query=(sum(increase(istio_request_duration_milliseconds_sum{destination_app='collector'}[1h]))/sum(increase(istio_request_duration_milliseconds_count{destination_app='collector'}[1h])))[1h:1s]


100 20844    0 20844    0     0   434k      0 --:--:-- --:--:-- --:--:--  442k
